In [36]:
%reload_ext autoreload
%autoreload
#%aimport

import numpy as np

from Electromagnetics.RecGridND import *

n_dim_space = 1

x0, x1, dx = None, None, None
n_dim = n_dim_space+1
assert n_dim>1
BC = 'IC'
make_refinements_random = False
S = 0.9
n_t = 300

##(t, x)
x0 = np.array([0.0, 0.0])
x1 = np.array([0.01, 10.0])
dx = np.array([x1[0]/1.9, 0.1])

assert dx[0]<S*dx[1]

    
rg_E = RGND(x0, x1, dx)
##refinements and setups
if make_refinements_random:
    rg = rg_E
    n_tot_0 = len(rg.cellsHier[0])
    n_ref = int(n_tot_0/5)

    cells_ref = {}

    import random
    cells_ref[0] = list(set([random.randint(0, n_tot_0-1) for i in range(n_ref)]))
    print('cells_ref: ', cells_ref)

    rg.NodesGetConnectedCells()
    rg.SetupBoundaryCellConnections()
    #cells_ref = rg.MarkAdditionalCellsForRefinement(cells_ref)
    cells_ref = rg.MarkAdditionalCellsForRefinement_PeriodicBoundary(cells_ref)
    print('cells_ref: ', cells_ref)

    rg.RefineCells(cells_ref)
    print('Refinement verification:', rg.VerifyCellsContinuity())

    n_tot_1 = len(rg.cellsHier[1])
    cells_ref = {0:[]}
    cells_ref[1] = list(set([random.randint(0, n_tot_1-1) for i in range(n_ref)]))
    print('cells_ref: ', cells_ref)

    rg.NodesGetConnectedCells()
    rg.SetupBoundaryCellConnections()
    #cells_ref = rg.MarkAdditionalCellsForRefinement(cells_ref)
    cells_ref = rg.MarkAdditionalCellsForRefinement_PeriodicBoundary(cells_ref)
    print('cells_ref: ', cells_ref)

    rg.RefineCells(cells_ref)
    print('Refinement verification:', rg.VerifyCellsContinuity())

    cells_ref = {}
    n_tot_0 = len(rg.cellsHier[0])
    n_tot_1 = len(rg.cellsHier[1])
    n_tot_2 = len(rg.cellsHier[2])
    cells_ref[0] = list(set([random.randint(0, n_tot_0-1) for i in range(n_ref)]))
    cells_ref[1] = list(set([random.randint(0, n_tot_1-1) for i in range(n_ref)]))
    cells_ref[2] = list(set([random.randint(0, n_tot_2-1) for i in range(n_ref)]))
    print('cells_ref: ', cells_ref)

    rg.NodesGetConnectedCells()
    rg.SetupBoundaryCellConnections()
    #cells_ref = rg.MarkAdditionalCellsForRefinement(cells_ref)
    cells_ref = rg.MarkAdditionalCellsForRefinement_PeriodicBoundary(cells_ref)
    print('cells_ref: ', cells_ref)

    rg.RefineCells(cells_ref)

rg_E.NodesGetConnectedCells()
rg_E.SetupBoundaryCellConnections()
assert rg_E.VerifyInternalAndBorderCellsContinuity()


rg_H = RGND(x0, x1, dx, rg=rg_E)
rg_H.NodesGetConnectedCells()
rg_H.SetupBoundaryCellConnections()
assert rg_H.VerifyInternalAndBorderCellsContinuity()



-- [  2 100] 200


In [37]:
from sympy import symbols, Function, Derivative, latex, sin, cos, lambdify, exp, tanh, sympify
from IPython.display import display, Math, Latex

t, z = symbols('t, z')
E = Function('E')(t, z)
H = Function('H')(t, z)
J_e = Function('J')(t, z)
J_m = Function('M')(t, z)

sigma_z = 2.0
z_mid = x1[1]/2
sigma_t=2.0
t_on = 2.0
t_off = 4.0
omega = 1.0
#J_e_val = exp(-sigma_z*(z-z_mid)**2)*(tanh(sigma_t*(t-t_on)) - tanh(sigma_t*(t-t_off)))*cos(omega*t)
J_e_val_t = (tanh(sigma_t*(t-t_on)) - tanh(sigma_t*(t-t_off)))*cos(omega*t)
J_e_val_z = exp(-sigma_z*(z-z_mid)**2)
J_e_val = J_e_val_t*J_e_val_z
J_m_val = 0.0

J_e_val_func = lambdify((t, z), J_e_val, 'numpy')

c = 1.0
eq_E = (Derivative(E, t) - Derivative(H, z) - J_e)#.subs(J_e, 0.0).simplify()
eq_H = (Derivative(H, t) - Derivative(E, z) + J_m).subs(J_m, 0.0).simplify()

print('differential equations:')
display(Math(latex(eq_E)))
display(Math(latex(eq_H)))

rg_E.AttachDiffEquations(eqs_list=[eq_E], vars_list=[E], pars_list=[J_e], indepVars_list=[t, z], pars_extern={H:rg_H})
rg_H.AttachDiffEquations(eqs_list=[eq_H], vars_list=[H], pars_list=[], indepVars_list=[t, z], pars_extern={E:rg_E})

rg_E.SetParsValues([[J_e_val_t, J_e_val_z]], pars_types=[ParType.seperable])
#rg_H.SetParsValues([J_m_val])

eq_E_parts = rg_E.DisintegrateEquations()
eq_H_parts = rg_H.DisintegrateEquations()
print('eq_E_parts:', eq_E_parts)
print('eq_H_parts:', eq_H_parts)


##-- boundary coditions
E_t0 = 0.0
#rg_H.AdvanceCornerInVarDir(t)
H_t0 = 0.0
rg_H.verbose=0

rg_E.ResetBoundaryConditions()
rg_E.AddBoundaryCondition({'expr':E-E_t0, 'dir':rg_E.IndepVarIndex[t], 'face':'n'})

rg_H.ResetBoundaryConditions()
rg_H.AddBoundaryCondition({'expr':H-H_t0, 'dir':rg_H.IndepVarIndex[t], 'face':'n'})


E_cc = []
E_cc.append({'der':0, 'dir':rg_E.IndepVarIndex[t], 'face':'n'})
rg_E.DefineContinuityCondsAcrossCellBorders({E:E_cc})

H_cc = []
H_cc.append({'der':0, 'dir':rg_H.IndepVarIndex[t], 'face':'n'})
rg_H.DefineContinuityCondsAcrossCellBorders({H:H_cc})


poly_order = [[1, 1]]
poly_order_max = [[0, 1]]
der_order_max = [[1, 0]]

rg_E.SetPolynomialBasisFuncs(poly_order)
rg_E.SetMaxPolyOrderToKeepForEachEq(poly_order_max)
rg_E.SetMaxDerivativeOrderForEachVar(der_order_max)
rg_H.SetPolynomialBasisFuncs(poly_order)
rg_H.SetMaxPolyOrderToKeepForEachEq(poly_order_max)
rg_H.SetMaxDerivativeOrderForEachVar(der_order_max)

rg_E.setupPolyStorageQues(10)
rg_H.setupPolyStorageQues(10)

der_order = [0, 0]


#test
#rg_E.SetEqIndicesToKeep(poly_order[0], poly_order_max[0])
#rg_E.TestSeperableTaylor(sin(t)*cos(z), [sin(t), cos(z)], printall=False)


##solve E at t0
x_res = rg_E.SetupMatrixEQs()
rg_E.SetPolyBasisCoeffs(x_res)

rg_E.SetDerivativeTemplatesForAllLevels([1, 1])


rg_E.GetCellBoundPolyCoeffOnFaceTemp_scaled_masked(poly_order=poly_order[0], vp_der_ord_max=der_order)
rg_E.RegisterICFieldsAtTheGivenSurface(v_ind=0, n_dir=0, face='p', der_order=der_order)
rg_E.RegisterFieldsAtTheGivenSurface(v_ind=0, n_dir=0, face='p', der_order=der_order)
#print('rg_E.FieldsAtSurf:', rg_E.FieldsAtSurf)
rg_E.AdvanceCornerInVarDir(t)


##solve H at t0
x_res = rg_H.SetupMatrixEQs()
rg_H.SetPolyBasisCoeffs(x_res)
rg_H.GetCellBoundPolyCoeffOnFaceTemp_scaled_masked(poly_order=poly_order[0], vp_der_ord_max=der_order)

rg_H.SetDerivativeTemplatesForAllLevels([1, 1])
rg_H.ResetPolyHier()

bcinds_cc = [0]
cc_equiv = [{'v_ind':0, 'dir':0, 'der':0, 'face':'n'}]

"""
figsize(10, 6)
n_pts_dim = [200]*n_dim
gslice=[{'dir':0, 'val':rg_E.W[0]}]
mesh, var_vals = rg_E.GetVarValuesOnMesh(0, n_pts_dim, gslice)
X, Y = mesh[0], var_vals
plot(X, Y)
_E_0_ = np.cos(X) + rg_E.W[0]
plot(X, _E_0_, 'r-.', linewidth=2)
#plot(X, Y/_E_0_, 'g-.')
grid()
show()
rg_E.polyHier[0] = rg_E.TakeDerivativePolyHier(rg_E.polyHier, v_ind=0, der_order=[0, 1])
mesh, var_vals = rg_E.GetVarValuesOnMesh(0, n_pts_dim, gslice)
X, Y = mesh[0], var_vals
plot(X, Y)
_d_E_0_ = -np.sin(X)
plot(X, _d_E_0_, 'r-.', linewidth=2)
grid()
show()
"""

def init_H():
    rg_H.AdvanceCornerInVarDir(t)
    x_res = rg_H.ResetRHSUseCCasIC_Leapfrog(bcinds_cc, cc_equiv, n_dir_lf=0)
    rg_H.SetPolyBasisCoeffs(x_res)
    rg_H.RegisterICFieldsAtTheGivenSurface(v_ind=0, n_dir=0, face='p', der_order=der_order)
    rg_H.RegisterFieldsAtTheGivenSurface(v_ind=0, n_dir=0, face='p', der_order=der_order)
    #rg_H.AdvanceCornerInVarDir(t)

def step_E():
    rg_E.PolyHierToMatRes(setSelfMatRes=True)
    rg_E.RegisterFieldsAtTheGivenSurface(v_ind=0, n_dir=0, face='p', der_order=der_order)
    rg_E.AdvanceCornerInVarDir(t)
    
    x_res = rg_E.ResetRHSUseCCasIC_Leapfrog(bcinds_cc, cc_equiv, n_dir_lf=0)
    rg_E.SetPolyBasisCoeffs(x_res)
    rg_E.RegisterFieldsAtTheGivenSurface(v_ind=0, n_dir=0, face='p', der_order=der_order)
    rg_E.AdvanceCornerInVarDir(t)

def step_H():
    rg_H.PolyHierToMatRes(setSelfMatRes=True)
    rg_H.RegisterFieldsAtTheGivenSurface(v_ind=0, n_dir=0, face='p', der_order=der_order)
    rg_H.AdvanceCornerInVarDir(t)
    
    x_res = rg_H.ResetRHSUseCCasIC_Leapfrog(bcinds_cc, cc_equiv, n_dir_lf=0)
    rg_H.SetPolyBasisCoeffs(x_res)
    rg_H.RegisterFieldsAtTheGivenSurface(v_ind=0, n_dir=0, face='p', der_order=der_order)
    rg_H.AdvanceCornerInVarDir(t)


init_H()

"""
print('np.max(np.abs(rg_H.mat_res)):', np.max(np.abs(rg_H.mat_res)))
gslice=[{'dir':0, 'val':rg_E.W[0]}]
mesh, var_vals = rg_E.GetVarValuesOnMesh(0, n_pts_dim, gslice)
X, Y = mesh[0], var_vals
plot(X, Y)
_E_0_ = _E_0_ + rg_E.W[0]
plot(X, _E_0_, 'r-.', linewidth=2)
#plot(X, _H_0_/Y, 'g')
grid()
show()

print('rg_H.x0:', rg_H.x0, 'rg_H.x1:', rg_H.x1, 'rg_H.W:', rg_H.W)
print('np.max(np.abs(rg_H.mat_res)):', np.max(np.abs(rg_H.mat_res)))
gslice=[{'dir':0, 'val':rg_H.W[0]}]
mesh, var_vals = rg_H.GetVarValuesOnMesh(0, n_pts_dim, gslice)
X, Y = mesh[0], var_vals
plot(X, Y)
_H_0_ = _d_E_0_*rg_H.W[0]
plot(X, _H_0_, 'r-.', linewidth=2)
#plot(X, _H_0_/Y, 'g')
grid()
show()
"""

#%pylab inline
#figsize(10, 6)
n_pts_dim = [200]*n_dim
gslice=[{'dir':0, 'val':0.0}]

X_E, Y_E = [None]*n_t, [None]*n_t
X_H, Y_H = [None]*n_t, [None]*n_t
for i in range(n_t):
    step_E()
    step_H()
    print(i, '  t:', rg_E.x1[0])
    
    mesh, var_vals = rg_E.GetVarValuesOnMesh(0, n_pts_dim, gslice)
    X_E[i], Y_E[i] = mesh[0], var_vals
    #plot(X, Y)
    mesh, var_vals = rg_H.GetVarValuesOnMesh(0, n_pts_dim, gslice)
    X_H[i], Y_H[i] = mesh[0], var_vals
    #plot(X, Y, 'r')
    #plot(X, J_e_val_func(rg_E.x1[0], X), 'g')
    #show()
    


differential equations:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

eq_E_parts: [[[[[1, 1, [t]]]]], [J(t, z) + Derivative(H(t, z), z)]]
eq_H_parts: [[[[[1, 1, [t]]]]], [Derivative(E(t, z), z)]]


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

self.mask_allp:
[[1 1]
 [1 1]
 [0 0]]
self.mask_xp:
[[0 0]
 [1 1]
 [0 0]]
self.mask_x_F:
[0, 0, 1, 1]
self.inds_0_allp:
[[ 0  1]
 [ 2  3]
 [-1 -1]]
self.inds_0_all_F:
[[0, (0, 0)], [1, (0, 1)], [2, (1, 0)], [3, (1, 1)]]
self.inds_keep:
[[1 1]
 [1 1]]
self.inds_keep__x_weight:
[[[0, 2, 1.0], [1, 3, 1.0]]]
------------------------------


<IPython.core.display.Math object>

/home/nima/ipython workspace/Electromagnetics/RecGridND.py:3459: ComplexWarning: Casting complex values to real discards the imaginary part
  p_f[inds_0_all_F_1[inds_0_FL_f0_dir[j]]] = PolyFaceCoeffs[lev][i][j]
/home/nima/ipython workspace/Electromagnetics/RecGridND.py:3496: ComplexWarning: Casting complex values to real discards the imaginary part
  p_f[inds_0_all_F_1[inds_0_FL_f0_dir[j]]] = PolyFaceCoeffs[lev][i][j]
/home/nima/ipython workspace/Electromagnetics/RecGridND.py:5598: ComplexWarning: Casting complex values to real discards the imaginary part
  var_vals[i] = self.GetVariableValueAtPointsInsideCell(var_ind, lev, c_ind, [pt_i], printit=printit)[0]


0   t: 0.04
1   t: 0.06
2   t: 0.08
3   t: 0.1
4   t: 0.12
5   t: 0.14
6   t: 0.16
7   t: 0.18
8   t: 0.2
9   t: 0.22
10   t: 0.24
11   t: 0.26
12   t: 0.28
13   t: 0.3
14   t: 0.32
15   t: 0.34
16   t: 0.36
17   t: 0.38
18   t: 0.4
19   t: 0.42
20   t: 0.44
21   t: 0.46
22   t: 0.48
23   t: 0.5
24   t: 0.52
25   t: 0.54
26   t: 0.56
27   t: 0.58
28   t: 0.6
29   t: 0.62
30   t: 0.64
31   t: 0.66
32   t: 0.68
33   t: 0.7
34   t: 0.72
35   t: 0.74
36   t: 0.76
37   t: 0.78
38   t: 0.8
39   t: 0.82
40   t: 0.84
41   t: 0.86
42   t: 0.88
43   t: 0.9
44   t: 0.92
45   t: 0.94
46   t: 0.96
47   t: 0.98
48   t: 1.0
49   t: 1.02
50   t: 1.04
51   t: 1.06
52   t: 1.08
53   t: 1.1
54   t: 1.12
55   t: 1.14
56   t: 1.16
57   t: 1.18
58   t: 1.2
59   t: 1.22
60   t: 1.24
61   t: 1.26
62   t: 1.28
63   t: 1.3
64   t: 1.32
65   t: 1.34
66   t: 1.36
67   t: 1.38
68   t: 1.4
69   t: 1.42
70   t: 1.44
71   t: 1.46
72   t: 1.48
73   t: 1.5
74   t: 1.52
75   t: 1.54
76   t: 1.56
77   t: 1.58
78   t: 1.6

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def animate(i):
    plt.clf()
    fig = plt.plot(X_E[i], Y_E[i], 'r')
    axes = plt.gca()
    #axes.set_xlim([xmin,xmax])
    axes.set_ylim([-1.0, 1.0])
    return fig
    
fig = plt.figure()
axes = plt.gca()
axes.set_ylim([-1.5, 1.5])
    
anim = animation.FuncAnimation(fig, animate, frames=n_t, interval=50)
plt.show()
    

In [14]:
plt.plot(X_E[10], Y_E[10])
plt.show()